1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [22]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [8]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [35]:
df.describe()

,BGGId,Rating
count,1.894215e+07,1.894215e+07
mean,1.099536e+05,7.130966e+00
std,9.280213e+04,1.543015e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [24]:
any_null_values = df.isnull().any()
any_null_values

#print("whole df/ per column:\n", (data == "?").sum(axis=0))
#print("whole df/ per column:\n", (df == "?").sum(axis=0))
null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [25]:
df = df.dropna()

In [26]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [27]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [34]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [32]:
duplicates.describe()

count     18942152
unique           1
top          False
freq      18942152
dtype: object

Filter dataset

In [52]:
rows_to_drop = df.loc[df['BGGId'] <= 10].index
rows_to_drop

# Ta bort dessa rader från DataFrame
#cleaned_df = df.drop(rows_to_drop)
#cleaned_df

Int64Index([  757584,   757585,   757586,   757587,   757588,   757589,
              757590,   757591,   757592,   757593,
            ...
            18677060, 18677061, 18677062, 18677063, 18677064, 18677065,
            18677066, 18677067, 18677068, 18677069],
           dtype='int64', length=53165)

In [69]:
game_ratings = 50

rating_counter = df['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df.drop(index=df[games_mask].index, inplace=True)

df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [63]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df.groupby('BGGId').size()
grouped_counts

BGGId
1          5352
2           562
3         15147
4           342
5         18387
          ...  
346703     1000
346965      253
347521       71
348955       67
349131       66
Length: 17552, dtype: int64

Train test split

In [71]:
# Define a function to split the data for each user
def user_split(data, test_size=0.2):
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size)
        train = train.append(train_user)
        test = test.append(test_user)
    
    return train, test

# Split the data
train_set, test_set = user_split(df, test_size=0.2)

print("Train Set:")
print(train_set)
print("\nTest Set:")
print(test_set)



/var/folders/f3/n44hzk017jg3wr6ylhv9l8lr0000gn/T/ipykernel_91836/1656574008.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(train_user)
/var/folders/f3/n44hzk017jg3wr6ylhv9l8lr0000gn/T/ipykernel_91836/1656574008.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test.append(test_user)
/var/folders/f3/n44hzk017jg3wr6ylhv9l8lr0000gn/T/ipykernel_91836/1656574008.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(train_user)
/var/folders/f3/n44hzk017jg3wr6ylhv9l8lr0000gn/T/ipykernel_91836/1656574008.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test.append(test_user)
/var/f

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

Här nedan skriver vi funktionen som tar input data, user, k, top_n

Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 